<a href="https://colab.research.google.com/github/adeotti/sudoku-env/blob/main/M9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# python version 3.9.11 
%pip install numpy==1.25.2 tensorboard==2.17.0 torchrl==0.4.0 gymnasium==0.29.1 tensordict==0.4.0

from IPython.display import clear_output
def clear():
  clear_output(wait=False)

import numpy as np
from dataclasses import dataclass
import math,sys
np.random.seed(42)
clear()

Environment

In [2]:
easy = np.array([
    [0, 0, 0, 5, 3, 1, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 3, 0, 1],
    [1, 0, 0, 8, 0, 0, 0, 0, 0],
    [0, 0, 4, 0, 0, 5, 6, 0, 0],
    [0, 0, 3, 9, 0, 2, 1, 4, 0],
    [6, 1, 5, 0, 7, 0, 0, 9, 8],
    [0, 2, 0, 0, 9, 6, 0, 1, 0],
    [0, 5, 7, 2, 0, 8, 0, 0, 6],
    [0, 6, 1, 7, 5, 3, 0, 2, 4]])

In [1]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Board_specs:
  size: tuple = (9,9)
  low: int = 1
  high: int = 9

def Board(specs=Board_specs):
  matrix = easy
  return matrix

def Region_Split(matrix):
  # Extract the 3X3 regions
    ZERO = 0 ; CONST = 3 ; MAX_COLUMN_END = 12
    row_start = col_start = ZERO
    row_end = col_end = CONST
    subgrid = []
    while len(subgrid) < len(matrix) :
        subgrid.append([row[col_start:col_end] for row in matrix[row_start:row_end]])
        col_start += CONST
        col_end += CONST
        if col_end == MAX_COLUMN_END :
            col_start = ZERO
            col_end = CONST
            row_start += CONST
            row_end += CONST
    assert len(subgrid) == len(matrix)
    subgrid = [np.concatenate(line) for line in subgrid]
    return subgrid

In [4]:
class Game:

    def __init__(self,action = None):
        self.board = Board()
        self.action = action

        self.modifiableCells = []
        
        for i,x in enumerate(self.board):
            for y in range(Board_specs.high):
                if x[y] == 0:
                    self.modifiableCells.append((i,y))            

    def Updated_board(self):
        if self.action is not None:
            self.action = self.action[0] if not len(self.action)== 3 else self.action
            row,column,value = self.action

            if (row,column) in self.modifiableCells:
                self.board[row][column] = value
                
        return self.board,self.action

In [5]:
class Util:
    @classmethod
    def Utilities(cls,gameinstance):
        if gameinstance is None :
            sys.exit(f"{cls.__name__} : No Game instance was provided")
        cls.upd_matrix,cls.action = gameinstance.Updated_board()

        # TODO : just transpose the matrix then ylist will be easy to implement
        cls.y_list = [[col[y_index] for col in cls.upd_matrix] for y_index in range(Board_specs.high)]
        cls.y = [line for line in cls.upd_matrix.T]
        cls.subgrid = Region_Split(cls.upd_matrix)
        cls.subgrid = [arr.tolist() for arr in cls.subgrid]

    @staticmethod
    def unique(lisst):
        return len(lisst) == len(set(lisst))

    @staticmethod
    def len_dif(lisst):
        return len(lisst) - len(set(lisst))

In [6]:
class boardConflicts: # Heuristic component

    previous_conflicts = 0

    def __init__(self,instance):
        Util.Utilities(instance)
        self.conflicts = 0
        self.previous_conflicts = 0
        self.reward_H = 0

    def count(self):
        for line in Util.upd_matrix:
            if not Util.unique(line) : self.conflicts += Util.len_dif(line)
        for col in Util.y_list:
            if not Util.unique(col) : self.conflicts += Util.len_dif(col)
        for submatrix in Util.subgrid:
            if not Util.unique(submatrix) : self.conflicts += Util.len_dif(submatrix)
        # H value
        if self.conflicts < self.previous_conflicts:
            self.reward_H = 2
        boardConflicts.previous_conflicts = self.conflicts
        return self.reward_H


In [7]:
class rewardFunction:
    # Reward function
    def __init__(self,instance):
        Util.Utilities(instance)
        self.action = Util.action
        self.matrix = Util.upd_matrix
        self.y_list = Util.y_list
        self.reward = 0
        if not len(self.action)==3: action = action[0]
        assert len(self.action)==3

    def rewardReturns(self):
        if self.action is None :
            sys.exit(f"{self.__class__.__name__} : Action is None")
        # x
        x = self.matrix[self.action[0]]; assert len(x) == Board_specs.high
        self.reward += 9 if (Util.len_dif(x) == 0) else - Util.len_dif(x)
        # y
        y_index = self.action[1]
        y = self.y_list[y_index]; assert len(y) == Board_specs.high
        self.reward += 9 if (Util.len_dif(y) == 0) else -Util.len_dif(y)
        # region
        n = int(math.sqrt(Board_specs.high))
        ix,iy = (self.action[0]//n)* n , (self.action[1]//n)* n
        region = np.concatenate(self.matrix[ix:ix+n , iy:iy+n])
        assert len(region) == Board_specs.high
        self.reward += 9 if (Util.len_dif(region) == 0) else  -Util.len_dif(region)
        return self.reward

In [8]:
def gameEnd(instance):
    Util.Utilities(instance)
    matrix = Util.upd_matrix
    y_list = Util.y_list
    subgrid = Util.subgrid
    all_uniquerow = all(Util.unique(line) for line in matrix)
    all_uniquecol = all(Util.unique(line) for line in y_list)
    all_unique_region = all(Util.unique(line) for line in subgrid)
    return all_uniquerow,all_uniquecol,all_unique_region

In [9]:
import gymnasium as gym
import gymnasium.spaces as spaces
from gymnasium.envs.registration import register

register(
    id="sudoku",
    entry_point="__main__:ENVI"
)

class ENVI(gym.Env):
    def __init__(self):
        super().__init__()
        self.action = None
        self.game = Game(None)
        self.updated,_ = self.game.Updated_board()



        self.size = Board_specs.high
        self.action_space = spaces.Tuple((spaces.Discrete(Board_specs.high),                        # x
                                          spaces.Discrete(Board_specs.high),                        # y
                                          spaces.Discrete(Board_specs.high,start=Board_specs.low))) # value, start = 1
        self.observation_space = spaces.Box(low=Board_specs.low,high=Board_specs.high,
                                            shape=Board_specs.size,dtype=float)

    def reset(self,seed = None) :
        super().reset(seed=seed)
        return np.float16(self.updated) ,{}

    def step(self,action):
        self.game = Game(action)
        self.updated,_ = Game(action).Updated_board()

        reward_base = rewardFunction(self.game).rewardReturns()
        heuristic = boardConflicts(self.game).count()
        reward = reward_base + heuristic
        terminated_list = gameEnd(self.game)
        terminated = (True if all(terminated_list) else False)

        info = {
            "conflicts" : heuristic
        }
        self.action = action
        return  np.float16(self.updated),reward,terminated,False,info

clear()

Training 

In [10]:
from tqdm import tqdm
from collections import deque

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import OneHotCategorical

from tensordict.nn import TensorDictModule
from tensordict import TensorDict

from torchrl.modules import ValueOperator,ProbabilisticActor
from torchrl.objectives.value import GAE
from torchrl.objectives import ClipPPOLoss

from torchrl.envs import TransformedEnv,GymEnv,Compose,DoubleToFloat,UnsqueezeTransform
# data collection and manipulation
from torchrl.collectors import SyncDataCollector
from torchrl.data import ReplayBuffer,SamplerWithoutReplacement,LazyTensorStorage

from torch.utils.tensorboard import SummaryWriter

l_rate = 0.01
sdg_momentum = 0.9

frames =  1000               # number of steps
sub_frame = 50              # for the most inner loop of the training step
total_frames = 200_000      # maximum steps
epochs = 10

gamma = 0.80
lmbda = 0.99
 

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# environnment
base = GymEnv("sudoku",device = device)
env = TransformedEnv(base,Compose(UnsqueezeTransform(in_keys=["observation"], unsqueeze_dim=0,allow_positive_dim=True),
                                  DoubleToFloat())
)
dummy_observation = env.reset()["observation"]
action_spec = tuple(env.action_spec.shape) # (3,9)
action_dist = env.action_spec.shape.numel() # 27
size = torch.flatten(dummy_observation).shape.numel() # 81

In [12]:
@torch.no_grad()
def weights_init(w):
  if isinstance(w,(nn.Conv2d,nn.LazyConv2d,nn.LazyLinear)):
    nn.init.kaiming_uniform(w.weight,mode="fan_in",nonlinearity="relu")
    if w.bias is not None : nn.init.zeros_(w.bias)

def Network_util(network : nn.Module):
  network.to(device)
  network.forward(dummy_observation)
  network.apply(weights_init)
  return network

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# environnment
base = GymEnv("sudoku",device = device)
env = TransformedEnv(base,Compose(UnsqueezeTransform(in_keys=["observation"], unsqueeze_dim=0,allow_positive_dim=True),
                                  DoubleToFloat())
)
dummy_observation = env.reset()["observation"]
action_spec = tuple(env.action_spec.shape) # (3,9)
action_dist = env.action_spec.shape.numel() # 27
size = torch.flatten(dummy_observation).shape.numel() # 81

In [14]:
# Actor
class ActorNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.size = size
    self.action_dist = action_dist
    self.action_spec = action_spec

    self.input_layer = nn.LazyLinear(81)
    self.flat = nn.Flatten()
    self.dense_one = nn.Linear(self.size,self.size)
    self.dense_two = nn.Linear(self.size,self.size)
    self.output = nn.Linear(self.size,self.action_dist)

  def forward(self,x):
    x = self.flat(x)
    x = F.relu(self.input_layer(x))
    #x = torch.flatten(x)
    x = F.relu(self.dense_one(x))
    x = F.relu(self.dense_two(x))
    x = F.relu(self.output(x))
    x = torch.unflatten(x,-1,(self.action_spec))
    return F.softmax(x,-1)

Network_util(ActorNetwork())
Actor = ActorNetwork()
Actor.load_state_dict(torch.load("100k_data/actor_100K.pth"))
PolicyModule = TensorDictModule(module=Actor, in_keys=["observation"],out_keys=["probs"])

PolicyModule = TensorDictModule(module=Actor, in_keys=["observation"],out_keys=["probs"])
PolicyModule = ProbabilisticActor(
  module=PolicyModule, spec=env.action_spec, in_keys=["probs"] ,
  distribution_class=OneHotCategorical, return_log_prob=True
)
Memory = ReplayBuffer(storage=LazyTensorStorage(max_size=frames),sampler=SamplerWithoutReplacement())
Collector = SyncDataCollector(create_env_fn=env,policy=PolicyModule,frames_per_batch=frames,total_frames=total_frames)
Collector.rollout()
clear()

In [15]:
# Critic
class ValueNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.size = size
    self.action_dist = action_dist
    self.action_spec = action_spec

    self.input_layer = nn.LazyLinear(self.size)
    self.flat = nn.Flatten()
    self.dense_one = nn.Linear(self.size,self.size)
    self.dense_two = nn.Linear(self.size,self.size)
    self.output = nn.Linear(self.size,1)

  def forward(self,x):
    x = self.flat(x)
    x = F.relu(self.input_layer(x))
    x = F.relu(self.dense_one(x))
    x = F.relu(self.dense_two(x))
    return self.output(x)

Critic = Network_util(network=ValueNetwork())

ValueModule = ValueOperator(module= Critic,in_keys=["observation"])
Advantage = GAE(gamma=gamma,lmbda=lmbda,value_network=ValueModule,average_gae=True,device=device)

In [16]:
loss = ClipPPOLoss(actor_network=PolicyModule,critic_network=ValueModule)
optimizer = torch.optim.SGD(params=loss.parameters(),lr=l_rate,momentum=sdg_momentum )

In [20]:
# @title Training
class Training:
    def __init__(self):
        self.policy = ActorNetwork()
        self.collector = Collector
        self.memory = Memory
        self.valuemodule = ValueModule
        self.advantage = Advantage
        self.lossfunction = loss
        self.optimizer = optimizer
        self.epochs = epochs

    def save_logs(self):
            log_dir = "/trainingData/"  
            self.writer = SummaryWriter(log_dir)

    def save_weight(self):
        path = "/trainingData/actor_200k.pth"  
        torch.save(self.policy.state_dict(),path)
    
    def fullyTrainedmodel(self):
        path = "/trainingData/fullyTrainedmodel.pth"

    def train(self,start : bool = None):
        if start:
            bestReward = -10
            rewardHistory = deque(maxlen = 10)
            self.save_logs()
            for i,data_tensordict in tqdm(enumerate(self.collector),total = total_frames/frames):

                for e in range(self.epochs):
                    dat = self.advantage(data_tensordict)
                    dat["advantage"] = dat["advantage"].unsqueeze(-1)
                    self.memory.extend(dat)

                    for n in range(total_frames//sub_frame):
                        subdata = Memory.sample(sub_frame)
                        loss_val = self.lossfunction(subdata.to(device))
                        loss_value = (loss_val["loss_objective"] + loss_val["loss_critic"] + loss_val["loss_entropy"])
                        reward = data_tensordict["next"]["reward"][0].mean().item()
                        loss_value.backward()
                        self.optimizer.step()
                        self.optimizer.zero_grad()

                self.writer.add_scalar("main/batch_number",i)
                self.writer.add_scalar("main/Advantage",dat["advantage"][0].item())
                self.writer.add_scalar("main/Loss_sum",loss_value.item())
                self.writer.add_scalar("main/reward",data_tensordict["next"]["reward"][0].mean().item())
                self.writer.add_scalar("main/reward_advantage",dat["next"]["reward"][0].mean().item())
                self.writer.add_scalar("loss/Loss_entropy",loss_val["loss_entropy"].item())
                self.writer.add_scalar("loss/Loss_critic",loss_val["loss_critic"].item())
                self.writer.add_scalar("loss/Loss_objective",loss_val["loss_objective"].item())

                currentReward = data_tensordict["next"]["reward"][0].mean()
                rewardHistory.append(currentReward)
                averageReward = sum(rewardHistory)/len(rewardHistory)

                if i % 10 == 0:
                  if averageReward > bestReward:
                    self.save_weight
                    bestReward = averageReward

                self.fullyTrainedmodel()
            


In [ ]:
Training().train(start=True)